In [1]:
# Set up graph-tool environment
!conda install -y -c conda-forge graph-tool --quiet 

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



In [2]:
import os
os.getcwd()


'/home/jovyan/diss-essay-3/Code'

In [3]:
## Set up packages, so that my notebook runs inside an enviornment where graph-tool is saved and operating

import pandas as pd 
import graph_tool.all as gt
import matplotlib.pyplot as plt
import numpy as np

print("graph-tool imported successfully!")

graph-tool imported successfully!


In [4]:
### Construct the June 2015 network ("professionally managed" only); using the same format as the looped files

np.random.seed(1234) # Set a seed to be used consistently between the different years' network constructions

sorted_062015 = pd.read_csv('../Data/Cleaned files/NRED HOA - 06 2015_subset.csv') 

sorted_062015

,Name,SOS#,Assn Type,# of Units,Telephone,Address1,Address2,City,State,Zip Code,County,Address2_Normalized,Firm_Normalized,Phone_Normalized,Firm_ByPhone,Firm_ByAddress,Firm_Final,Name_Changed_Flag
0,1200 Riverside Drive Association,283-1972,REG,100,(702) 737-8580,"Firstservice Residential, Nevada, Llc",8290 Arville St,Las Vegas,NV,89139,Washoe,8290 arville st,firstservice residential nevada,7.027379e+09,firstservice residential nevada,firstservice residential nevada,firstservice residential nevada,False
1,391 Willow Court Homeowners Association,5719-1978,REG,4,(775) 831-5345,Swiss Time Management,Po Box 5742,Incline Village,NV,89450,Washoe,po box 5742,swiss time management,7.758315e+09,swiss time management,swiss time management,swiss time management,False
2,609 Lariat Circle Homeowners Association,3992-1986,REG,4,(775) 832-6604,Incline Property Management,848 Tanager St Ste M,Incline Village,NV,89451,Washoe,848 tanager st,incline property management,7.758327e+09,incline property management,incline property management,incline property management,False
3,999 Lakeshore Association,1270-1967,REG,60,(775) 832-0284,Incline Property Management,848 Tanager St Ste M,Incline Village,NV,89451,Washoe,848 tanager st,incline property management,7.758320e+09,incline property management,incline property management,incline property management,False
4,Aberdeen @ Southern Highlands Homeowners Assoc...,11900-2001,SUB,88,(702) 458-2580,Colonial Property Management,8595 S Eastern Ave,Las Vegas,NV,89123,Clark,8595 s eastern ave,colonial property management,7.024583e+09,colonial property management,colonial property management,colonial property management,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,York Village Community Association,0679632006-4,REG,127,(702) 655-7064,Las Vegas Valley Community Management,7571 Tule Springs Rd,Las Vegas,NV,89131,Clark,7571 tule spring rd,las vega valley community management,7.026557e+09,las vega valley community management,las vega valley community management,las vega valley community management,False
2685,Yorkshire Manor Association,2411-1971,REG,160,(775) 674-8000,Kenyon & Associates,645 Sierra Rose Dr Ste 105A,Reno,NV,89511,Washoe,645 sierra rose dr,kenyon associate,7.756748e+09,kenyon associate,kenyon associate,kenyon associate,False
2686,Yorkshire Manor No.2 Association,3248-1972,REG,108,(775) 674-8000,Kenyon & Associates,645 Sierra Rose Dr Ste 105A,Reno,NV,89511,Washoe,645 sierra rose dr,kenyon associate,7.756748e+09,kenyon associate,kenyon associate,kenyon associate,False
2687,Zephyr Ridge Homeowners' Association,0553902013-5,REG,88,(702) 531-3382,Camco,Po Box 12117,Las Vegas,NV,89112,Clark,po box 12117,camco,7.025313e+09,camco,camco,camco,False


In [5]:
## Visualizations

unique_companies_062015 = sorted_062015['Firm_Final'].unique()  # Pull out the individual management company names, changed to final
total_nv_network_062015 = gt.Graph()  # Create an empty graph to populate with the data

# Any decisions regarding which managed ones to keep in (e.g. individually registered) and how to label by size/type?

hoa_vertices_062015 = {} # Set empty dictionaries with the respective nodes
company_vertices_062015 = {}

company_name_prop_062015 = total_nv_network_062015.new_vertex_property("string")
company_city_prop_062015 = total_nv_network_062015.new_vertex_property("string")
company_state_prop_062015 = total_nv_network_062015.new_vertex_property("string") 

for company_062015 in unique_companies_062015: # Sort through the management companies, store their information, and add as vertices to the graph
    company_data_062015 = sorted_062015[sorted_062015['Firm_Final'] == company_062015]
    company_vertex = total_nv_network_062015.add_vertex()
    company_vertices_062015[company_062015] = company_vertex
    company_name_prop_062015[company_vertex] = company_062015

    city = company_data_062015.iloc[0]['City']  
    state = company_data_062015.iloc[0]['State']  
    company_city_prop_062015[company_vertex] = city  
    company_state_prop_062015[company_vertex] = state 
    
    for index, row in company_data_062015.iterrows(): # Iterate through the company data, add the unique HOAs (which are now rows in the transformed data), and draw edges between the node types
        hoa_name = row['Name']
        if hoa_name not in hoa_vertices_062015:
            hoa_vertex = total_nv_network_062015.add_vertex()
            hoa_vertices_062015[hoa_name] = hoa_vertex
        else:
            hoa_vertex = hoa_vertices_062015[hoa_name]
        total_nv_network_062015.add_edge(hoa_vertex, company_vertex)

pos = gt.sfdp_layout(total_nv_network_062015, K=120, C=0.01) # Space out the nodes using the force-directed algorithm

# Color and sizing
vertex_color = total_nv_network_062015.new_vertex_property("vector<double>") # Assign color scheme based on HOA or management (and latter's location)

for v in total_nv_network_062015.vertices():
    if v in company_vertices_062015.values():
        city = company_city_prop_062015[v]
        state = company_state_prop_062015[v]
        if state != "NV":
            vertex_color[v] = (0.5, 0, 0.5, 0.75)  # Purple color for companies with address outside NV
        elif city in ["Boulder City", "Henderson", "Las Vegas", "Mesquite", "North Las Vegas", "Blue Diamond", "Bunkerville", "Cal-Nev-Ari", "Enterprise", "Goodsprings", "Indian Springs", "Laughlin", "Moapa Town", "Moapa Valley", "Mount Charleston", "Nelson", "Paradise", "Sandy Valley", "Searchlight", "Spring Valley", "Summerlin South", "Sunrise Manor", "Whitney", "Winchester"]:
            vertex_color[v] = (0, 1, 0, 0.75)  # Green color for Vegas MSA-headquartered companies
        else:
            vertex_color[v] = (1, 0, 0, 0.75)  # Red color for other cities in NV
    else:
        vertex_color[v] = (0, 0, 1, 0.75)  # Blue color for all individual HOAs, inferring membership
        
# Graphic creation and save
gt.graph_draw(total_nv_network_062015, pos, vertex_fill_color=vertex_color, output_size=(2000, 2000), output="../Output/062015_managed.png") # Draw and save the graphic to the project folder

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7fb07c886410, at 0x7fb07c88fe90>

In [6]:
## Statistics (for professionally managed HOAs)

# Degree centrality: add up total number of edges (representing company-HOA connections), sort by the most, and calculate proportion
degrees = total_nv_network_062015.degree_property_map('total')
total_degree_centrality_sum = total_nv_network_062015.num_edges()
top_10 = sorted(company_vertices_062015.values(), key=lambda v: degrees[v], reverse=True)[:10]
print("Top 10 Vertices by Degree Centrality - June 2015:")
for i, vertex in enumerate(top_10):
    company_name = company_name_prop_062015[vertex]
    degree_centrality = degrees[vertex]
    proportion_of_total = (degree_centrality / total_degree_centrality_sum)*100
    print(f"{i+1}. {company_name}, Number of HOAs: {degree_centrality}, Proportion of Total: {proportion_of_total:.1f}")

# Average community size by firm (in units): group by company, add up number of units responsible for and number of HOAs, and sort by the most
company_stats_062015 = sorted_062015.groupby('Firm_Final').agg({'# of Units': 'sum', 'Name': 'count'})
company_stats_062015['Average Community Size'] = company_stats_062015['# of Units'] / company_stats_062015['Name']
company_stats_062015['Average Community Size'] = company_stats_062015['Average Community Size'].round()
top_10_average_community_size = company_stats_062015.sort_values(by='Average Community Size', ascending=False).head(10)
print("Top 10 Companies by Average Community Size - June 2015:")
print(top_10_average_community_size)

Top 10 Vertices by Degree Centrality - June 2015:
1. firstservice residential nevada, Number of HOAs: 339, Proportion of Total: 12.6
2. camco, Number of HOAs: 225, Proportion of Total: 8.4
3. terra west management service, Number of HOAs: 166, Proportion of Total: 6.2
4. associa, Number of HOAs: 157, Proportion of Total: 5.8
5. colonial property management, Number of HOAs: 116, Proportion of Total: 4.3
6. taylor association management, Number of HOAs: 98, Proportion of Total: 3.6
7. community management group, Number of HOAs: 87, Proportion of Total: 3.2
8. thoroughbred management, Number of HOAs: 82, Proportion of Total: 3.0
9. mesa management, Number of HOAs: 63, Proportion of Total: 2.3
10. nicklin community management service, Number of HOAs: 63, Proportion of Total: 2.3
Top 10 Companies by Average Community Size - June 2015:
                                             # of Units  Name  \
Firm_Final                                                      
coyote spring investment    

In [7]:
### Construct the June 2015 network (all)
### Should appear the same but with freestanding yellow nodes for non-managed (under my definition)

In [8]:
### Construct the June 2025 network ("professionally managed" only)

# Necessary packages are already loaded in

np.random.seed(1234)

sorted_062025 = pd.read_csv('../Data/Cleaned files/NRED HOA - 06 2025_subset.csv')  
sorted_062025

,Name,SOS#,Assn Type,# of Units,Telephone,Address1,Address2,City,State,Zip Code,County,Address2_Normalized,Firm_Normalized,Phone_Normalized,Firm_ByPhone,Firm_ByAddress,Firm_Final,Name_Changed_Flag
0,"14Th Street Duplex Homeowners Association, Inc",12771462021-4,REG,2.0,(702) 290-2122,"14Th Street Duplex Homeowners Association, Inc",9292 Bluemist Falls St,Las Vegas,NV,89123,Clark,9292 bluemist fall st,14th street duplex homeowner association,7.022902e+09,14th street duplex homeowner association,14th street duplex homeowner association,14th street duplex homeowner association,False
1,317 Quaking Aspen Homeowners Association,0100782019-8,SUB,8.0,(775) 284-4434,"Western Nevada Management, Inc","255 W. Peckham Ln., Ste 2",Reno,NV,89509,Douglas,255 w peckham ln,western nevada management,7.752844e+09,western nevada management,western nevada management,western nevada management,False
2,360 Galaxy Homeowners Association,13220372021-8,SUB,8.0,(775) 460-9662,Kamehame Group Llc,Po Box 10193,Zephyr Cove,NV,89448,Douglas,po box 10193,kamehame group,7.754610e+09,kamehame group,kamehame group,kamehame group,False
3,"389 Cottonwood Homeowners Association, A Nevad...",30976-2003,REG,2.0,(716) 861-5230,Gian Properties,4221 Transit Road,Williamsville,NY,14221,NaN,4221 transit rd,gian propertie,7.168615e+09,gian propertie,gian propertie,gian propertie,False
4,"428 Quaking Aspen Hoa, Llc",0617562006-1,SUB,4.0,(775) 690-0808,"428 Quaking Aspen Hoa, Llc",16310 Jeffe Ct,Reno,NV,89511,Washoe,16310 jeffe ct,428 quaking aspen hoa,7.756901e+09,428 quaking aspen hoa,428 quaking aspen hoa,428 quaking aspen hoa,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,York Village Community Association,0679632006-4,REG,127.0,(702) 655-7064,Nevada Community Management,9500 West Flamingo Rd Ste 204,Las Vegas,NV,89147,Clark,9500 w flamingo rd,nevada community management,7.026557e+09,nevada community management,nevada community management,nevada community management,False
3456,Yorkshire Manor Association,2411-1971,REG,160.0,(775) 284-4434,"Western Nevada Management, Inc",255 W Peckham Ln Ste 2,Reno,NV,89509,Washoe,255 w peckham ln,western nevada management,7.752844e+09,western nevada management,western nevada management,western nevada management,False
3457,Yorkshire Manor No.2 Association,3248-1972,REG,108.0,(775) 446-4479,Hoamco,5484 Reno Corporate Drive #100,Reno,NV,89511,Washoe,5484 reno corporate drive 100,hoamco,7.754464e+09,hoamco,hoamco,hoamco,False
3458,Zephyr Ridge Homeowners' Association,0553902013-5,REG,88.0,(702) 362-6262,Terra West Management Services,6655 S Cimarron Rd Ste 200,Las Vegas,NV,89113,Clark,6655 s cimarron rd,terra west management service,7.023626e+09,terra west management service,terra west management service,terra west management service,False


In [9]:
## Visualizations

unique_companies_062025 = sorted_062025['Firm_Final'].unique()  
total_nv_network_062025 = gt.Graph()  

hoa_vertices_062025 = {} 
company_vertices_062025 = {}

# Any decisions regarding which managed ones to keep in (e.g. individually registered) and how to label by size/type?

company_name_prop_062025 = total_nv_network_062025.new_vertex_property("string")
company_city_prop_062025 = total_nv_network_062025.new_vertex_property("string")
company_state_prop_062025 = total_nv_network_062025.new_vertex_property("string") 

for company_062025 in unique_companies_062025: 
    company_data_062025 = sorted_062025[sorted_062025['Firm_Final'] == company_062025]
    company_vertex = total_nv_network_062025.add_vertex()
    company_vertices_062025[company_062025] = company_vertex
    company_name_prop_062025[company_vertex] = company_062025

    city = company_data_062025.iloc[0]['City']  
    state = company_data_062025.iloc[0]['State']  
    company_city_prop_062025[company_vertex] = city  
    company_state_prop_062025[company_vertex] = state 
    
    for index, row in company_data_062025.iterrows(): 
        hoa_name = row['Name']
        if hoa_name not in hoa_vertices_062025:
            hoa_vertex = total_nv_network_062025.add_vertex()
            hoa_vertices_062025[hoa_name] = hoa_vertex
        else:
            hoa_vertex = hoa_vertices_062025[hoa_name]
        total_nv_network_062025.add_edge(hoa_vertex, company_vertex)

pos = gt.sfdp_layout(total_nv_network_062025, K=50) 

# Color and sizing
vertex_color = total_nv_network_062025.new_vertex_property("vector<double>") 

for v in total_nv_network_062025.vertices():
    if v in company_vertices_062025.values():
        city = company_city_prop_062025[v]
        state = company_state_prop_062025[v]
        if state != "NV":
            vertex_color[v] = (0.5, 0, 0.5, 0.75)  # Purple color for companies with address outside NV
        elif city in ["Boulder City", "Henderson", "Las Vegas", "Mesquite", "North Las Vegas", "Blue Diamond", "Bunkerville", "Cal-Nev-Ari", "Enterprise", "Goodsprings", "Indian Springs", "Laughlin", "Moapa Town", "Moapa Valley", "Mount Charleston", "Nelson", "Paradise", "Sandy Valley", "Searchlight", "Spring Valley", "Summerlin South", "Sunrise Manor", "Whitney", "Winchester"]:
            vertex_color[v] = (0, 1, 0, 0.75)  # Green color for Vegas MSA-headquartered companies
        else:
            vertex_color[v] = (1, 0, 0, 0.75)  # Red color for other cities in NV
    else:
        vertex_color[v] = (0, 0, 1, 0.75)  # Blue color for all individual HOAs
        
# Graphic creation and save
gt.graph_draw(total_nv_network_062025, pos, vertex_fill_color=vertex_color, output_size=(1000, 1000), output="../Output/062025_managed.png") # Draw and save the graphic to the project folder

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7fb07ccb1510, at 0x7fb07d7cee90>

In [10]:
## Statistics (for professionally managed HOAs)

# Degree centrality
degrees = total_nv_network_062025.degree_property_map('total')
total_degree_centrality_sum = total_nv_network_062025.num_edges()
top_10 = sorted(company_vertices_062025.values(), key=lambda v: degrees[v], reverse=True)[:10]
print("Top 10 Vertices by Degree Centrality - June 2025:")
for i, vertex in enumerate(top_10):
    company_name = company_name_prop_062025[vertex]
    degree_centrality = degrees[vertex]
    proportion_of_total = (degree_centrality / total_degree_centrality_sum)*100
    print(f"{i+1}. {company_name}, Number of HOAs: {degree_centrality}, Proportion of Total: {proportion_of_total:.1f}")

# Average community size by firm 
company_stats_062025 = sorted_062025.groupby('Firm_Final').agg({'# of Units': 'sum', 'Name': 'count'})
company_stats_062025['Average Community Size'] = company_stats_062025['# of Units'] / company_stats_062025['Name']
company_stats_062025['Average Community Size'] = company_stats_062025['Average Community Size'].round()
top_10_average_community_size = company_stats_062025.sort_values(by='Average Community Size', ascending=False).head(10)
print("Top 10 Companies by Average Community Size - June 2025:")
print(top_10_average_community_size)

Top 10 Vertices by Degree Centrality - June 2025:
1. firstservice residential nevada, Number of HOAs: 288, Proportion of Total: 8.3
2. associa, Number of HOAs: 209, Proportion of Total: 6.0
3. camco, Number of HOAs: 209, Proportion of Total: 6.0
4. terra west management service, Number of HOAs: 204, Proportion of Total: 5.9
5. nevada community management, Number of HOAs: 173, Proportion of Total: 5.0
6. colonial community management, Number of HOAs: 161, Proportion of Total: 4.7
7. the management trust, Number of HOAs: 148, Proportion of Total: 4.3
8. taylor association management, Number of HOAs: 133, Proportion of Total: 3.8
9. thoroughbred management, Number of HOAs: 127, Proportion of Total: 3.7
10. prime community management, Number of HOAs: 122, Proportion of Total: 3.5
Top 10 Companies by Average Community Size - June 2025:
                                           # of Units  Name  \
Firm_Final                                                    
coyote spring investment       

In [ ]:
### Construct the June 2025 network (all)

In [ ]:
## Rewiring (going to need to adjust to perform in parallel, as well as use an adjacency matrix)

# Step 1: Log transform the actual structure's degree values
logged_degrees_actual = np.log(total_nv_network_062025.degree_property_map('total').a)

# Step 2: Build the random rewiring function (only changing edges from HOAs to other possible companies, not each other), iterate and check progress, and fill in list of results
def rewire_edges(graph):
    edges = list(graph.edges())
    np.random.shuffle(edges)
    for edge in edges:
        source, target = edge
        source_degree = graph.degree_property_map('total')[source]
        target_degree = graph.degree_property_map('total')[target]
        if source_degree == 1 and target_degree > 1:
            possible_targets = [v for v in graph.vertices() if v!= source and graph.degree_property_map('total')[v] > 1 and not graph.edge(source, v)]
            new_target = np.random.choice(possible_targets)
            graph.remove_edge(edge)
            graph.add_edge(source, new_target)
    return graph

iterations = 100 # Can play around with different amounts
rewired_networks = []
for i in range(iterations):
    print(f"Iteration {i+1}/{iterations}")
    rewired_network = rewire_edges(total_nv_network_062025.copy())
    rewired_networks.append(rewired_network)
    
# Step 3: Log transform the rewirings' degree values and calculate the mean and SD of them sample population we have created
logged_degrees_rewired = []
for rewired_network in rewired_networks:
    degrees_rewired = rewired_network.degree_property_map('total').a
    logged_degrees_rewired.append(np.log(degrees_rewired))

mean_logged_degrees_rewired = np.mean(logged_degrees_rewired, axis=0)
std_logged_degrees_rewired = np.mean(logged_degrees_rewired, axis=0)

# Step 4: Calculate the z-scores for the rewirings using above information
z_scores = []
for degrees in logged_degrees_rewired:
    z_score = (degrees - mean_logged_degrees_rewired) / std_logged_degrees_rewired
    z_scores.append(z_score)

# Step 5: Calculate the actual structure's z-scores using above information
actual_z_score = (logged_degrees_actual - - mean_logged_degrees_rewired) / std_logged_degrees_rewired

# Step 6: Plot the results

In [ ]:
# Looping and tracking clusters across entire time period (refer to the g-t package dictionary for clues)

In [1]:
## HHI analysis
import pandas as pd
import glob
import os
import re

# Total

input_folder = "../Data/Cleaned files/"
output_folder = "../Data/Misc"
output_csv = os.path.join(output_folder, "HHI_by_file.csv")
firm_column = "Firm_Final"
unit_column = "# of Units"

pattern = re.compile(r"NRED HOA - (\d{2}) (\d{4})_subset\.csv")

results = []

for file in glob.glob(os.path.join(input_folder, "NRED HOA - *_subset.csv")):
    
    filename = os.path.basename(file)
    match = pattern.match(filename)

    if not match:
        continue

    month = int(match.group(1))
    year = int(match.group(2))

    df = pd.read_csv(file)

    df = df.dropna(subset=[firm_column])

    df[unit_column] = pd.to_numeric(df[unit_column], errors="coerce")

    # Total
    firm_counts = df[firm_column].value_counts()
    total_associations = firm_counts.sum()

    assoc_shares = (firm_counts / total_associations) * 100
    hhi_assoc = (assoc_shares ** 2).sum()
    
    # Weighted by firms
    df_units = df.dropna(subset=[unit_column])

    firm_units = df_units.groupby(firm_column)[unit_column].sum()
    total_units = firm_units.sum()

    unit_shares = (firm_units / total_units) * 100
    hhi_units = (unit_shares ** 2).sum()

    # Create file
    results.append({
        "file_name": filename,
        "year": year,
        "month": month,
        "num_associations": int(total_associations),
        "num_firms": firm_counts.shape[0],
        "total_units": int(total_units),
        "HHI_association_weighted": hhi_assoc,
        "HHI_unit_weighted": hhi_units
    })

hhi_df = pd.DataFrame(results)
hhi_df = hhi_df.sort_values(["year", "month"]).reset_index(drop=True)

hhi_df.to_csv(output_csv, index=False)

print(f"Saved sequential HHI results to {output_csv}")

# LV, Reno, and other listings

input_folder = "../Data/Cleaned files/"
output_folder = "../Data/Misc"
os.makedirs(output_folder, exist_ok=True)

firm_column = "Firm_Final"
unit_column = "# of Units"
city_column = "City"

pattern = re.compile(r"NRED HOA - (\d{2}) (\d{4})_subset\.csv")

# Define MSAs
lv_msa_cities = ["Boulder City", "Henderson", "Las Vegas", "Mesquite", "North Las Vegas", "Blue Diamond", "Bunkerville", "Cal-Nev-Ari", 
                 "Enterprise", "Goodsprings", "Indian Springs", "Laughlin", "Moapa Town", "Moapa Valley", 
                 "Mount Charleston", "Nelson", "Paradise", "Sandy Valley", "Searchlight", "Spring Valley", 
                 "Summerlin South", "Sunrise Manor", "Whitney", "Winchester"]
reno_msa_cities = ["Reno", "Sparks", "Carson City", "Fallon", "Fernley", "Yerington", "Cold Springs", "Crystal Bay", 
                   "Empire", "Gerlach", "Golden Valley", "Incline Village", "Lemmon Valley", "Mogul", "Nixon","Spanish Springs", "Sun Valley", 
                   "Sutcliffe", "Verdi", "Virginia City", "Wadsworth"]

results = {"LV": [], "Reno": [], "Other": []}

for file in glob.glob(os.path.join(input_folder, "NRED HOA - *_subset.csv")):
    
    filename = os.path.basename(file)
    match = pattern.match(filename)

    if not match:
        continue

    month = int(match.group(1))
    year = int(match.group(2))

    df = pd.read_csv(file)
    df = df.dropna(subset=[firm_column])
    df[unit_column] = pd.to_numeric(df[unit_column], errors="coerce")
    df[city_column] = df[city_column].str.title().str.strip()  # normalize

    # Assign regions
    regions = {
        "LV": df[df[city_column].isin(lv_msa_cities)],
        "Reno": df[df[city_column].isin(reno_msa_cities)],
        "Other": df[~df[city_column].isin(lv_msa_cities + reno_msa_cities)]
    }

    for region_name, df_region in regions.items():
        if df_region.empty:
            continue

        # Association-weighted HHI
        firm_counts = df_region[firm_column].value_counts()
        total_associations = firm_counts.sum()
        assoc_shares = (firm_counts / total_associations) * 100
        hhi_assoc = (assoc_shares ** 2).sum()

        # Unit-weighted HHI
        firm_units = df_region.groupby(firm_column)[unit_column].sum()
        total_units = firm_units.sum()
        unit_shares = (firm_units / total_units) * 100
        hhi_units = (unit_shares ** 2).sum()

        results[region_name].append({
            "file_name": filename,
            "year": year,
            "month": month,
            "num_associations": int(total_associations),
            "num_firms": firm_counts.shape[0],
            "total_units": int(total_units),
            "HHI_association_weighted": hhi_assoc,
            "HHI_unit_weighted": hhi_units
        })

for region_name, df_list in results.items():
    if df_list:
        hhi_df = pd.DataFrame(df_list).sort_values(["year", "month"]).reset_index(drop=True)
        output_csv = os.path.join(output_folder, f"HHI_by_file_{region_name}.csv")
        hhi_df.to_csv(output_csv, index=False)
        print(f"Saved {region_name} HHI file to {output_csv}")

# Construct summary file

output_folder = "../Data/Misc"
regions = ["Total", "LV", "Reno", "Other"]

hhi_region_dfs = {}

for region in regions:
    path = os.path.join(output_folder, f"HHI_by_file_{region}.csv") if region != "Total" else os.path.join(output_folder, "HHI_by_file.csv")
    if os.path.exists(path):
        df = pd.read_csv(path)
        # Keep only year, month, HHI, total associations, and total units
        df = df[["year", "month", "HHI_association_weighted", "HHI_unit_weighted", "num_associations", "total_units"]]
        # Rename columns to include region
        df = df.rename(columns={
            "HHI_association_weighted": f"HHI_{region}_assoc",
            "HHI_unit_weighted": f"HHI_{region}_units",
            "num_associations": f"num_associations_{region}",
            "total_units": f"total_units_{region}"
        })
        hhi_region_dfs[region] = df

hhi_summary = None
for region, df in hhi_region_dfs.items():
    if hhi_summary is None:
        hhi_summary = df
    else:
        hhi_summary = pd.merge(hhi_summary, df, on=["year", "month"], how="outer")

# Sort chronologically
hhi_summary = hhi_summary.sort_values(["year", "month"]).reset_index(drop=True)

# Save summary
summary_csv = os.path.join(output_folder, "HHI_summary_all_regions.csv")
hhi_summary.to_csv(summary_csv, index=False)
print(f"Saved summary HHI file (with totals) to {summary_csv}")

Saved sequential HHI results to ../Data/Misc/HHI_by_file.csv
Saved LV HHI file to ../Data/Misc/HHI_by_file_LV.csv
Saved Reno HHI file to ../Data/Misc/HHI_by_file_Reno.csv
Saved Other HHI file to ../Data/Misc/HHI_by_file_Other.csv
Saved summary HHI file (with totals) to ../Data/Misc/HHI_summary_all_regions.csv
